<a href="https://colab.research.google.com/github/mridul-eecs/Named_Entity_Recognition/blob/main/transformer_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install spacy-transformers
# annot: https://tecoholic.github.io/ner-annotator/

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [4]:
def spacy_docbin(file= 'train.json'):
  f = open(file)
  TRAIN_DATA = json.load(f)

  for text, annot in tqdm(TRAIN_DATA['annotations']): 
      doc = nlp.make_doc(text) 
      ents = []
      for start, end, label in annot["entities"]:
          span = doc.char_span(start, end, label=label, alignment_mode="contract")
          if span is None:
              print("Skipping entity")
          else:
              ents.append(span)
      doc.ents = ents 
      db.add(doc)
  db.to_disk("./{}.spacy".format(file.split('.')[0]))
spacy_docbin('train.json')
spacy_docbin('test.json')

100%|██████████| 8/8 [00:00<00:00, 2269.18it/s]


In [5]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --gpu --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: GPU
- Transformer: roberta-base
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
! python -m spacy train config.cfg --gpu-id 0 --output ./ --paths.train ./train.spacy --paths.dev ./test.spacy 

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-02 14:34:39,990] [INFO] Set up nlp object from config
[2022-08-02 14:34:40,001] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-02 14:34:40,005] [INFO] Created vocabulary
[2022-08-02 14:34:40,006] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 483kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 5.04MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 3.29MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 7.60MB/s]
Downloading: 100% 478M/478M [00:06<00:00, 74.1MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with anoth

In [9]:
nlp_ner = spacy.load("model-best")
doc = nlp_ner("users by division")
spacy.displacy.render(doc, style="ent", jupyter=True)
